In [ ]:
for text in llm("AI is going to", stream=True):
    print(text, end="", flush=True)


# results on the cpu using ctransformers
### - 27.4 seconds
### output:
<p>
have an enormous impact on the future of the industry.
This year, AI was all over the show floor and there are a number of companies that are using AI in their products. Here's a quick look at some of them:
Fulcrum is an automated, artificial intelligence-powered software solution that helps manufacturers improve productivity and efficiency by automating processes, removing errors, and freeing up employees to focus on more complex tasks.
Kapil Khera, co-founder and CEO of Fulcrum, told Inman: "I don't know a single company that doesn't have AI in their product. I think it's the most important thing to come out of 2018."
The company is currently working on machine learning tools and workflow applications with machine intelligence technology so you can really begin building these powerful AI-powered solutions. The full AI-focused ecosystem, with automation in product design and sophisticated analytics, will be available to all customers by the end of 2018.
Kapil Khera, co-founder and CEO of Fulcrum
Surely
</p>

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)


In [ ]:

def get_response_tiny_llama(text):
    path='/teamspace/studios/this_studio/TinyLlama-1.1B-Chat-v1.0-GPTQ'
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
    prompt = text
    system_message = "You are a story writing assistant"
    prompt_template=f'''<|system|>
    {system_message}</s>
    <|user|>
    {prompt}</s>
    <|assistant|>
    '''

    print("\n\n*** Generate:")

    input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
    output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
    return(tokenizer.decode(output[0]))

In [ ]:
print(get_response_tiny_llama("what is qlora?"))

# result on gpu 2.8 seconds

# handling the embedding model

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)


In [51]:
import os
pdf_list=os.listdir("/Users/matansharon/python/opensource_rag/pdf_files")
text_list=os.listdir("/Users/matansharon/python/opensource_rag/text_files")

for pdf in pdf_list:
    temp=pdf.replace(".pdf",".txt")
    if temp not in text_list:
        text=convert_from_pdf_to_text("pdf_files/"+pdf)
        with open("text_files/"+temp, 'w') as f:
            f.write(text)

TypeError: write() argument must be str, not list

In [52]:
def convert_from_pdf_to_text(file_path):
    import PyPDF2

    read_pdf = PyPDF2.PdfReader(file_path)
    pages = read_pdf.pages
    res=''
    for page in pages:
        res+=page.extract_text()
    return res
pages=convert_from_pdf_to_text('/Users/matansharon/python/opensource_rag/pdf_files/A semantic loss for ontology classification.pdf')


In [ ]:
import chromadb
client=chromadb.Client()
import os
import chromadb.utils.embedding_functions as embedding_functions
huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key=os.environ.get("HuggingFace_API_KEY"),
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

collection=client.get_or_create_collection('test',embedding_function=huggingface_ef)
